In [5]:

from __future__ import print_function
from __future__ import division

import os

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from scipy.optimize import curve_fit

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

def run(clf):
    observations = pd.read_csv(os.path.join('./', 'training_set_observations.csv'), index_col=0)
    nest_counts = pd.read_csv(
                os.path.join('./', 'training_set_nest_counts.csv'),
                index_col=[0,1]
              )
    def preprocess_timeseries(timeseries, first_year, fillna_value=0):
        """ Takes one of the timeseries dataframes, removes
            columns before `first_year`, and fills NaN values
            with the preceeding value. Then backfills any
            remaining NaNs.

            As a courtesy, also turns year column name into
            integers for easy comparisons.
        """
        # column type
        timeseries.columns = timeseries.columns.astype(int)

        # subset to just data after first_year
        timeseries = timeseries.loc[:, timeseries.columns >= first_year]

        # Forward fill count values. This is a strong assumption.
        timeseries.fillna(method="ffill", axis=1, inplace=True)
        timeseries.fillna(method="bfill", axis=1, inplace=True)

        # For sites with no observations, fill with fill_na_value
        timeseries.fillna(fillna_value, inplace=True)

        return timeseries
    
    def preprocess_timeseries_interpolation(timeseries, first_year, fillna_value=0):
        """ Takes one of the timeseries dataframes, removes
            columns before `first_year`, and fills NaN values
            with the preceeding value. Then backfills any
            remaining NaNs.

            As a courtesy, also turns year column name into
            integers for easy comparisons.
        """
        # column type
        timeseries.columns = timeseries.columns.astype(int)

        # subset to just data after first_year
        timeseries = timeseries.loc[:, timeseries.columns >= first_year]

        # interpolate then extrapolate using scipy optimize
        timeseries.interpolate(method='linear',order=2,inplace=True)
        def func(x, a, b, c, d):
            return a * (x ** 3) + b * (x ** 2) + c * x + d
        guess = (0.5, 0.5, 0.5, 0.5)
        fit_df = timeseries.dropna()
        col_params = {}
        # Curve fit each column
        for col in fit_df.columns:
            # Get x & y
            x = fit_df.index.astype(float).values
            y = fit_df[col].values
            # Curve fit column and get curve parameters
            params = curve_fit(func, x, y, guess)
            # Store optimized parameters
            col_params[col] = params[0]
        for col in timeseries.columns:
            # Get the index values for NaNs in the column
            x = timeseries[pd.isnull(timeseries[col])].index.astype(float).values
            # Extrapolate those points with the fitted function
            timeseries.loc[col,x] = func(x, *col_params[col])

        # For sites with no observations, fill with fill_na_value
        timeseries.fillna(fillna_value, inplace=True)

        return timeseries

    nest_counts = preprocess_timeseries(nest_counts,
                                        1980,
                                        fillna_value=0.0)

    e_n_values = pd.read_csv(
                 os.path.join('./', 'training_set_e_n.csv'),
                 index_col=[0,1]
             )

    # Process error data to match our nest_counts data
    e_n_values = preprocess_timeseries(e_n_values, 1980, fillna_value=0.05)

    def amape(y_true, y_pred, accuracies):
        """ Adjusted MAPE
        """
        not_nan_mask = ~np.isnan(y_true)

        # calculate absolute error
        abs_error = (np.abs(y_true[not_nan_mask] - y_pred[not_nan_mask]))

        # calculate the percent error (replacing 0 with 1
        # in order to avoid divide-by-zero errors).
        pct_error = abs_error / np.maximum(1, y_true[not_nan_mask])

        # adjust error by count accuracies
        adj_error = pct_error / accuracies[not_nan_mask]

        # return the mean as a percentage
        return np.mean(adj_error)
    
    
    def train_model_per_row(ts, acc, split_year=2010):
        # Split into train/test to tune our parameter
        train = ts.iloc[ts.index < split_year]

        test = ts.iloc[ts.index >= split_year]
        test_acc = acc.iloc[acc.index >= split_year]

        # Store best lag parameter
        best_mape = np.inf 
        best_lag = None

        # Test linear regression models with the most recent
        # 2 points through using all of the points
        for lag in range(2, train.shape[0],2):
            # fit the model
            temp_model = clf()
            temp_model.fit(
                train.index[-lag:].values.reshape(-1, 1),
                train[-lag:]
            )

            # make our predictions on the test set
            preds = temp_model.predict(
                        test.index.values.reshape(-1, 1)
                    )

            # calculate the score using the custom metric
            mape = amape(test.values,
                         preds,
                         test_acc.values)

            # if it's the best score yet, hold on to the parameter
            if mape < best_mape:
                best_mape = mape
                best_lag = lag

        # return model re-trained on entire dataset
        final_model = clf()
        final_model.fit(
            ts.index[-best_lag:].values.reshape(-1, 1),
            ts[-best_lag:]
        )

        return best_mape, final_model

    models = {}
    scores = []

    for i, row in tqdm_notebook(nest_counts.iterrows(),
                                total=nest_counts.shape[0]):
        acc = e_n_values.loc[i]
        score, models[i] = train_model_per_row(row, acc)
        scores.append(score)
        
    submission_format = pd.read_csv(
        os.path.join('./','submission_format.csv'),
        index_col=[0, 1]
    )
    
    print("mape: ", np.array(scores).mean())
    
    preds = []

    # For every row in the submission file
    for i, row in tqdm_notebook(submission_format.iterrows(),
                                total=submission_format.shape[0]):

        # get the model for this site + common_name
        model = models[i]

        # make predictions using the model
        row_predictions = model.predict(
            submission_format.columns.values.reshape(-1, 1)
        )

        # keep our predictions, rounded to nearest whole number
        preds.append(np.round(row_predictions))

    # Create a dataframe that we can write out to a CSV
    prediction_df = pd.DataFrame(preds,
                                 index=submission_format.index,
                                 columns=submission_format.columns)
    
    return prediction_df


from sklearn.linear_model import LinearRegression as lr
from sklearn.ensemble import GradientBoostingRegressor as gbr
from sklearn.ensemble import ExtraTreesRegressor as etr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.svm import SVR, NuSVR
from sklearn.kernel_ridge import KernelRidge as krr
from sklearn.linear_model import Lasso as lasso

gb = run(gbr)
lin = run(lr)
et = run(etr)
rf = run(rfr)

defaults = (lin + et + gb + rf) / 4
    


mape:  52.07213995369804


mape:  57.71358932596669


mape:  52.07207728235157


mape:  55.617344901274336



In [6]:
from __future__ import print_function
from __future__ import division

import os

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from scipy.optimize import curve_fit

# let's not pollute this blog post with warnings
from warnings import filterwarnings
filterwarnings('ignore')

def interp_withlag(clf):
    observations = pd.read_csv('training_set_observations.csv', index_col=0)
    observations.head()

    nest_counts = pd.read_csv(
                    'training_set_nest_counts.csv',
                    index_col=[0,1]
                  )

    # get a sort order for the sites with the most observations
    sorted_idx = (pd.notnull(nest_counts)
                    .sum(axis=1)
                    .sort_values(ascending=False)
                    .index)


    def preprocess_timeseries(timeseries, first_year, fillna_value=0):
        """ Takes one of the timeseries dataframes, removes
            columns before `first_year`, and fills NaN values
            with the preceeding value. Then backfills any
            remaining NaNs.

            As a courtesy, also turns year column name into
            integers for easy comparisons.
        """
        # column type
        timeseries.columns = timeseries.columns.astype(int)

        # subset to just data after first_year
        timeseries = timeseries.loc[:, timeseries.columns >= first_year]

        # Forward fill count values. This is a strong assumption.
        timeseries.fillna(method="ffill", axis=1, inplace=True)
        timeseries.fillna(method="bfill", axis=1, inplace=True)

        # For sites with no observations, fill with fill_na_value
        timeseries.fillna(fillna_value, inplace=True)

        return timeseries

    def preprocess_timeseries_interpolation_perrow(timeseries, first_year, fillna_value=0):
        """ Takes one of the timeseries dataframes, removes
            columns before `first_year`, and fills NaN values
            with the preceeding value. Then backfills any
            remaining NaNs.

            As a courtesy, also turns year column name into
            integers for easy comparisons.
        """
        # column type
        timeseries.columns = timeseries.columns.astype(int)
        def interp(xxx):
            xx = xxx.values
            if np.isnan(xx).sum() == xx.shape[0]:
                xxx.values[:] = fillna_value
                return xxx
            mask = np.isnan(xx)
            xx[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), xx[~mask])
            xxx.values[:] = xx
            return xxx

        # subset to just data after first_year
        timeseries = timeseries.loc[:, timeseries.columns >= first_year]
        timeseries = timeseries.apply(lambda x: interp(x),axis=1)
        timeseries.fillna(fillna_value, inplace=True)
        return timeseries

    nest_counts = preprocess_timeseries_interpolation_perrow(nest_counts,
                                        1980,
                                        fillna_value=0.0)
    nest_counts.head()

    e_n_values = pd.read_csv(
                      'training_set_e_n.csv',
                     index_col=[0,1]
                 )

    # Process error data to match our nest_counts data
    e_n_values = preprocess_timeseries(e_n_values, 1980, fillna_value=0.05)
    e_n_values.head()

    def amape(y_true, y_pred, accuracies):
        """ Adjusted MAPE
        """
        not_nan_mask = ~np.isnan(y_true)

        # calculate absolute error
        abs_error = (np.abs(y_true[not_nan_mask] - y_pred[not_nan_mask]))

        # calculate the percent error (replacing 0 with 1
        # in order to avoid divide-by-zero errors).
        pct_error = abs_error / np.maximum(1, y_true[not_nan_mask])

        # adjust error by count accuracies
        adj_error = pct_error / accuracies[not_nan_mask]

        # return the mean as a percentage
        return np.mean(adj_error)

    from sklearn.linear_model import LinearRegression as lin
    from sklearn.ensemble import GradientBoostingRegressor as gbr
    from sklearn.ensemble import ExtraTreesRegressor as etr
    from sklearn.ensemble import RandomForestRegressor as rfr
    from sklearn.svm import SVR, NuSVR
    from sklearn.kernel_ridge import KernelRidge as krr
    from sklearn.linear_model import Lasso as lasso

    def train_model_per_row(ts, acc, split_year=2010):
        # Split into train/test to tune our parameter
        train = ts.iloc[ts.index < split_year]

        test = ts.iloc[ts.index >= split_year]
        test_acc = acc.iloc[acc.index >= split_year]

        # Store best lag parameter
        best_mape = np.inf 
        best_lag = None

        # Test linear regression models with the most recent
        # 2 points through using all of the points
        for lag in range(2, train.shape[0]):
            # fit the model
            temp_model = clf()
            temp_model.fit(
                train.index[-lag:].values.reshape(-1, 1),
                train[-lag:]#,epochs=50,verbose=False, batch_size=1, shuffle=False
            )

            # make our predictions on the test set
            preds = temp_model.predict(
                        test.index.values.reshape(-1, 1)
                    )

            # calculate the score using the custom metric
            mape = amape(test.values,
                         preds,
                         test_acc.values)

            # if it's the best score yet, hold on to the parameter
            if mape < best_mape:
                best_mape = mape
                best_lag = lag
    #     print( best_lag," : ",best_mape)
        # return model re-trained on entire dataset
        final_model =  clf()
        final_model.fit(
            ts.index[-best_lag:].values.reshape(-1, 1),
            ts[-best_lag:]#,epochs=50,verbose=False, batch_size=1, shuffle=False
        )

        return best_mape, final_model

    models = {}
    scores = []

    # print(nest_counts.head())

    for i, row in tqdm_notebook(nest_counts.iterrows(),
                                total=nest_counts.shape[0]):
        acc = e_n_values.loc[i]
        score, models[i] = train_model_per_row(row, acc)
        scores.append(score)

    print("average amape: ",np.array(scores).ravel().mean())    

    submission_format = pd.read_csv(
        'submission_format.csv',
        index_col=[0, 1]
    )

    print(submission_format.shape)
    submission_format.head()

    preds = []

    # For every row in the submission file
    for i, row in tqdm_notebook(submission_format.iterrows(),
                                total=submission_format.shape[0]):

        # get the model for this site + common_name
        model = models[i]

        # make predictions using the model
        row_predictions = model.predict(
            submission_format.columns.values.reshape(-1, 1)
        )

        # keep our predictions, rounded to nearest whole number
        preds.append(np.round(row_predictions))

    # Create a dataframe that we can write out to a CSV
    prediction_df = pd.DataFrame(preds,
                                 index=submission_format.index,
                                 columns=submission_format.columns)

    prediction_df.head()

    return prediction_df


lin = interp_withlag(lr)
et = interp_withlag(etr)
gb = interp_withlag(gbr)
rf = interp_withlag(rfr)
svr = interp_withlag(SVR)
lso = interp_withlag(lasso)
kr = interp_withlag(krr)

interp_sub = (lin + et + gb + rf + svr + lso + kr) / 7






average amape:  6.315300735219814
(648, 4)


average amape:  3.1677187670461473
(648, 4)


average amape:  3.1674773535736365
(648, 4)


average amape:  3.442255283916547
(648, 4)


average amape:  4.80327511026785
(648, 4)


average amape:  6.197934556394347
(648, 4)


average amape:  4.82496849597852
(648, 4)



In [7]:
sub = (interp_sub + defaults) / 2


In [8]:
sub.to_csv('./submission.csv')